# 0 导入必要的包

In [1]:
import numpy as np
import pandas as pd
import random
import sys
import os
from tqdm import tqdm
sys.path.append("../../tools/")
import commontools
import funclib
import time

%load_ext autoreload

# 1. 读取数据

In [56]:
# 1515 数据
sprot1515 = pd.read_csv('./data/ecoli1515/uniprot1515.tsv',  sep='\t')
sprot1515['isemzyme'] = sprot1515.ec_number.apply(lambda x: True if str(x)!='nan' else False)
sprot1515['isMultiFunctional'] = sprot1515.ec_number.apply(lambda x: True if ';' in str(x) else False)

In [63]:
# 酶与非酶数据
train = pd.read_hdf('../task1/data/train.h5',key='data')
train.ec_number = train.ec_number.apply(lambda x: x.replace(',', ''))
test = pd.read_hdf('../task1/data/test.h5',key='data')
test.ec_number = test.ec_number.apply(lambda x: x.replace(',', ''))
head = funclib.table_head + ['f'+str(i) for i in range(1, 1901) ]
train.columns = head
test.columns = head

In [71]:
base=pd.concat([train,test], axis=0,  ignore_index=True)

In [72]:
base

,id,name,isemzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,...,f1891,f1892,f1893,f1894,f1895,f1896,f1897,f1898,f1899,f1900
0,P02802,MT1_MOUSE,False,False,1,-,0,1986-07-21,1986-07-21,2021-04-07,...,-0.002212,0.119389,-0.019647,-0.016905,-0.532317,0.008659,-0.011296,0.001098,0.794472,0.210124
1,P03706,ENLYS_LAMBD,True,False,1,4.2.2.,4,1986-07-21,1986-07-21,2021-04-07,...,-0.067741,-0.060251,-0.087261,0.011515,-0.034108,-0.051713,-0.096768,0.025106,0.029159,-0.005789
2,P00806,ENLYS_BPT7,True,False,1,3.5.1.28,4,1986-07-21,2007-01-23,2021-04-07,...,0.045758,0.105044,-0.074721,0.057162,-0.063020,0.073825,-0.014284,-0.209938,-0.147285,-0.015436
3,P00643,T2H2_HAEPH,True,False,1,3.1.21.4,4,1986-07-21,1986-07-21,2020-12-02,...,-0.048356,0.058379,-0.157282,0.014700,-0.140881,0.522770,-0.015979,0.020724,-0.129453,0.008047
4,P01044,KNG1_BOVIN,False,False,1,-,0,1986-07-21,1986-07-21,2021-02-10,...,0.026815,-0.193191,0.009667,0.009924,-0.029288,0.021616,-0.024972,0.009794,0.074014,-0.134112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445793,Q06GJ0,HEXC_OSTFU,True,False,1,3.2.1.52,4,2021-04-07,2006-10-31,2021-04-07,...,-0.008599,-0.014624,-0.730082,0.850414,-0.070828,-0.001001,-0.014013,-0.191038,-0.024860,0.031504
445794,O17679,SET6_CAEEL,True,True,2,2.1.1.- 2.1.1.366,3,2021-04-07,2002-10-01,2021-04-07,...,0.034749,0.076677,-0.110957,0.018792,-0.130642,-0.059171,-0.069531,0.008016,-0.126152,0.018104
445795,A0A193AU77,GGT24_PUNGR,True,True,6,2.4.1.136 2.4.1.120 2.4.1.170 2.4.1.177 2.4.1....,4,2021-04-07,2016-10-05,2021-04-07,...,0.016355,0.042844,-0.034745,0.018539,-0.111699,-0.190519,0.011879,0.042398,0.448878,0.110567
445796,Q7PRG3,HKT_ANOGA,True,True,2,2.6.1.63 2.6.1.44,4,2021-04-07,2007-01-09,2021-04-07,...,0.011857,0.333768,-0.477595,0.105536,-0.026960,-0.105675,-0.020688,0.018101,-0.017889,0.605568


In [73]:
aa = base.merge(sprot1515, on='id', how='right')

In [83]:
aa[aa.date_sequence_update.isnull()]

,id,name,isemzyme_x,isMultiFunctional_x,functionCounts,ec_number_x,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,...,f1895,f1896,f1897,f1898,f1899,f1900,ec_number_y,seq_y,isemzyme_y,isMultiFunctional_y
4,P56100,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,7.1.1.7,MWYFAWILGTLLACSFGVITALALEHVESGKAGQEDI,True,False
8,P0DTT0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.6.5.-,MIEKLRNIAIIAHVDHGKTTLVDKLLQQSGTFDSRAETQERVMDSN...,True,False
9,Q46865,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.1.-.-,MEKRTPHTRLSQVKKLVNAGQVRTTRSALLNADELGLDFDGMCNVI...,True,False
10,P0AE70,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.1.27.-,MVSRYVPDMGDLIWVDFDPTKGSEQAGHRPAVVLSPFMYNNKTGMC...,True,False
11,P0AG24,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.7.6.5; 3.1.7.2,MYLFESLNQLIQTYLPEDQIKRLRQAYLVARDAHEGQTRSSGEPYI...,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,P76561,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.1.-.-,MKHDHFVVQSPDKPAQQLLLLFHGVGDNPVAMGEIGNWFAPLFPDA...,True,False
4410,P0AE06,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MNKNRGFTPLAVVLMLSGSLALTGCDDKQAQQGGQQMPAVGVVTVK...,False,False
4411,P0A6D0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MRSSAKQEELVKAFKALLKEEKFSSQGEIVAALQEQGFDNINQSKV...,False,False
4419,P0ABN9,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MLFTIQLIIILICLFYGARKGGIALGLLGGIGLVILVFVFHLQPGK...,False,False


In [61]:
sprot1515

,id,ec_number,seq,isemzyme,isMultiFunctional
0,P42641,3.6.5.-,MKFVDEASILVVAGDGGNGCVSFRREKYIPKGGPDGGDGGDGGDVW...,True,False
1,P0AE22,3.1.3.2,MRKITQAISAVCLLFALNSSAVALASSPSPLNPGTNVARLAEQAPI...,True,False
2,P21499,3.1.13.1,MSQDPFQEREAEKYANPIPSREFILEHLTKREKPASRDELAVELHI...,True,False
3,P39286,3.6.1.-,MSKNKLSKGQQRRVNANHQRRLKTSKEKPDYDDNLFGEPDEGIVIS...,True,False
4,P56100,7.1.1.7,MWYFAWILGTLLACSFGVITALALEHVESGKAGQEDI,True,False
...,...,...,...,...,...
4422,P36837,NaN,MNTTTPMGMLQQPRPFFMIFFVELWERFGYYGVQGVLAVFFVKQLG...,False,False
4423,P77304,NaN,MSTANQKPTESVSLNAFKQPKAFYLIFSIELWERFGYYGLQGIMAV...,False,False
4424,P27248,2.1.2.10,MAQQTPLYEQHTLCGARMVDFHGWMMPLHYGSQIDEHHAVRTDAGM...,True,False
4425,P0AC88,4.2.1.47,MSKVALITGVTGQDGSYLAEFLLEKGYEVHGIKRRASSFNTERVDH...,True,False


In [43]:
np.isnan(sprot1515[sprot1515.ec_number.isnull()].ec_number.values[0])

True

In [60]:
sprot1515[sprot1515['isMultiFunctional']]

,id,ec_number,seq,isemzyme,isMultiFunctional
6,P23837,2.7.13.3; 3.1.3.-,MKKLLRLFFPLSLRVRFLLATAAVVLVLSLAYGMVALIGYSVSFDK...,True,True
11,P0AG24,2.7.6.5; 3.1.7.2,MYLFESLNQLIQTYLPEDQIKRLRQAYLVARDAHEGQTRSSGEPYI...,True,True
94,P0A840,3.1.3.5; 3.1.3.6; 3.6.1.11,MRILLSNDDGVHAPGIQTLAKALREFADVQVVAPDRNRSGASNSLT...,True,True
105,P76422,2.7.1.49; 2.7.4.7,MKRINALTIAGTDPSGGAGIQADLKTFSALGAYGCSVITALVAQNT...,True,True
132,P37769,1.1.1.127; 1.1.1.-,MILSAFSLEGKVAVVTGCDTGLGQGMALGLAQAGCDIVGINIVEPT...,True,True
...,...,...,...,...,...
4262,P0A9M5,2.4.2.-; 2.4.2.22,MSEKYIVTWDMLQIHARKLASRLMPSEQWKGIIAVSRGGLVPGALL...,True,True
4263,P09099,2.7.1.17; 2.7.1.-,MYIGIDLGTSGVKVILLNEQGEVVAAQTEKLTVSRPHPLWSEQDPE...,True,True
4373,P45470,3.1.2.-; 3.5.1.-; 3.5.1.124,MSKKIAVLITDEFEDSEFTSPADEFRKAGHEVITIEKQAGKTVKGK...,True,True
4376,Q46845,1.8.4.-; 1.11.1.-,MTDNTYQPAKVWTWDKSAGGAFANINRPVSGPTHEKTLPVGKHPLQ...,True,True


In [90]:
aa = sprot1515[sprot1515.seq.apply(lambda x:len(x)).sort_values()<50]

<ipython-input-90-5179d5f58d27>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  aa = sprot1515[sprot1515.seq.apply(lambda x:len(x)).sort_values()<50]


In [93]:
aa[~aa.ec_number.isnull()]

,id,ec_number,seq,isemzyme,isMultiFunctional
4,P56100,7.1.1.7,MWYFAWILGTLLACSFGVITALALEHVESGKAGQEDI,True,False


In [94]:
abase = pd.read_csv('./data/ecoli1515/ecoprot_seq.csv')
abase.head(3)

,blattner_id,seq
0,b0001,MKRISTTITTTITITTGNGAG
1,b0002,MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...
2,b0003,MVKVYAPASSANMSVGFDVLGAAVTPVDGALLGDVVTVEAAETFSL...


In [96]:
bbase= pd.read_csv('./data/ecoli1515/ecogene_information.csv')
bbase.head(3)

,blattner_id,ENZYME
0,b0001,NaN
1,b0002,1.1.1.3;2.7.2.4
2,b0003,2.7.1.39


In [97]:
cbase = abase.merge(bbase, how='left', on='blattner_id')

In [98]:
cbase

,blattner_id,seq,ENZYME
0,b0001,MKRISTTITTTITITTGNGAG,NaN
1,b0002,MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...,1.1.1.3;2.7.2.4
2,b0003,MVKVYAPASSANMSVGFDVLGAAVTPVDGALLGDVVTVEAAETFSL...,2.7.1.39
3,b0004,MKLYNLKDHNEQVSFAQAVTQGLGKNQGLFFPHDLPEFSLTEIDEM...,4.2.3.1
4,b0005,MKKMQSIVLALSLVLVAPMAAQAAEITLVPSVKLQIGDRDNRGYYW...,NaN
...,...,...,...
4317,b4706,MKPALRDFIAIVQERLASVTA,NaN
4318,b4708,ENTNGLIRQYFPKKTCLAQYTQHELDLVAAQLNNRPRKTLKFKTPK...,NaN
4319,b4709,MASVSISCPSCSATDGVVRNGKSTAGHQRYLCSHCRKTWQLQFTYT...,NaN
4320,b4710,MPGNSPHYGRWPQHDFTSLKKLRPQSVTSRIQPGSDVIVCAEMDEQ...,NaN


In [100]:
dbase = sprot1515.merge(cbase, how='left', on='seq')

In [102]:
dbase.columns=['id','ec_number', 'seq', 'isemzyme','isMultiFunctional','blattner_id', 'blattner_ec_number']

In [106]:
dbase.to_csv('./data/ecoli1515/1515testset.tsv', index=None, sep='\t')